命令式编程（Imperative Programming）：<br>
- 定义：<br>
    - 命令式编程通过编写一系列明确的指令（即“命令”）来描述程序应如何一步步改变内部状态以完成任务。<br>
- 特点：<br>
    - 关注“怎么做”：开发者需要明确控制程序中状态的变更和执行流程。<br>
    - 常用语言：C、C++、Python（普通代码）、Java 等。<br>
    - 代码通常由语句和控制流结构（如循环、条件分支）构成。<br>
- 例子：<br>
    - 逐步修改变量以计算结果，每行代码都指示计算机执行具体操作


符号式编程（Symbolic Programming）：<br>
- 定义：<br>
    - 符号式编程（或声明式编程的一种形式）关注“做什么”而非“怎么做”，主要描述问题的结构或计算图，延迟执行具体计算。<br>
- 特点：<br>
    - 不直接执行具体运算，而是构建一个表达计算过程的符号图，待整张图构建完成后，再统一执行。
    - 常见于许多深度学习框架（如早期的 TensorFlow 1.x 或 Theano），其中开发者构建一个静态的计算图。
    - 易于进行全局优化和分布式计算，但调试和灵活性上可能稍逊。
- 例子：<br>
    - 定义张量之间的操作关系构成计算图，在调用 session.run() 时统一计算结果，而不是一步步执行。

混合式编程（Hybrid Programming）：<br>
- 定义：<br>
    - 混合式编程将命令式和符号式的优点相结合，既允许程序员在构建符号计算图时获得优化和自动求导的优势，又能通过命令式语句灵活地调试和控制流程。<br>
- 特点：<br>
    - 灵活性好：既可以像命令式那样逐步调试，也可以构建静态计算图以进行全局优化。<br>
    - 许多现代深度学习库（如 TensorFlow Eager 模式、PyTorch）采用这种方式：默认提供命令式（或动态图）运行，同时也支持导出为静态图进行优化和部署。<br>
- 例子：<br>
    - 在 PyTorch 中，默认以命令式方式编写代码，但也可以借助 TorchScript 将动态图转换为静态计算图，从而利用符号式优化和跨平台部署的优势。

目前为止，本书主要关注的是命令式编程（imperative programming）。 命令式编程使用诸如print、“+”和if之类的语句来更改程序的状态。 考虑下面这段简单的命令式程序：

In [2]:
def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g

print(fancy_func(1, 2, 3, 4))

10


Python是一种解释型语言（interpreted language）。因此，当对上面的fancy_func函数求值时，它按顺序执行函数体的操作。也就是说，它将通过对e = add(a, b)求值，并将结果存储为变量e，从而更改程序的状态。接下来的两个语句f = add(c, d)和g = add(e, f)也将执行类似地操作，即执行加法计算并将结果存储为变量

Python 在运行时通过解释器将高级代码即时翻译执行，而不是编译成独立的可执行文件，所以它被归类为解释型语言

尽管命令式编程很方便，但可能效率不高。一方面原因，Python会单独执行这三个函数的调用，而没有考虑add函数在fancy_func中被重复调用。如果在一个GPU（甚至多个GPU）上执行这些命令，那么Python解释器产生的开销可能会非常大。此外，它需要保存e和f的变量值，直到fancy_func中的所有语句都执行完毕。这是因为程序不知道在执行语句e = add(a, b)和f = add(c, d)之后，其他部分是否会使用变量e和f。

考虑另一种选择符号式编程（symbolic programming），即代码通常只在完全定义了过程之后才执行计算。这个策略被多个深度学习框架使用，包括Theano和TensorFlow（后者已经获得了命令式编程的扩展）。一般包括以下步骤：

定义计算流程；
1. 将流程编译成可执行的程序；
2. 给定输入，调用编译好的程序执行。

这将允许进行大量的优化。首先，在大多数情况下，我们可以跳过Python解释器。从而消除因为多个更快的GPU与单个CPU上的单个Python线程搭配使用时产生的性能瓶颈。其次，编译器可以将上述代码优化和重写为print((1 + 2) + (3 + 4))甚至print(10)。因为编译器在将其转换为机器指令之前可以看到完整的代码，所以这种优化是可以实现的。例如，只要某个变量不再需要，编译器就可以释放内存（或者从不分配内存），或者将代码转换为一个完全等价的片段。下面，我们将通过模拟命令式编程来进一步了解符号式编程的概念。

“模拟命令式编程”是指提前构造好一个完整的程序（即所有操作步骤的集合），然后一次性将其编译并执行，而不是逐行直接调用函数。这样可以展示如何构建一个“命令式程序”的完整表达，再由解释器执行。这在一些深度学习框架中也体现为将动态图导出为静态计算图进行整体编译与运行的思想。

In [3]:
def add_():
    # 返回一个多行字符串
    # 该字符串包含了一个函数的定义
    return '''
def add(a, b):
    return a + b
'''

def fancy_func_():
    return '''
def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
'''

def evoke_():
    # 负责组装一整个程序代码，因此返回的完整字符串解释一个完整的程序
    return add_() + fancy_func_() + 'print(fancy_func(1, 2, 3, 4))'

prog = evoke_()
print(prog)
# 使用 Python 内置的 compile 函数将整个程序字符串 prog 编译成一个可执行的代码对象。
y = compile(prog, '', 'exec')
# 使用 exec 函数执行编译后的代码对象 y
exec(y)

# 先以字符串的形式构造完整的程序（包含函数定义和调用），
# 然后将程序整体编译并执行。


def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
print(fancy_func(1, 2, 3, 4))
10


要了解混合式编程的工作原理，最简单的方法是考虑具有多层的深层网络。按照惯例，Python解释器需要执行所有层的代码来生成一条指令，然后将该指令转发到CPU或GPU。对于单个的（快速的）计算设备，这不会导致任何重大问题。另一方面，如果我们使用先进的8-GPU服务器，比如AWS P3dn.24xlarge实例，Python将很难让所有的GPU都保持忙碌。在这里，瓶颈是单线程的Python解释器。让我们看看如何通过将Sequential替换为HybridSequential来解决代码中这个瓶颈。首先，我们定义一个简单的多层感知机。

Python 解释器通常是单线程执行的，这意味着它一次只能执行一条指令，不能同时调度多个任务。这就导致：

- 数据调度瓶颈： 单线程的 Python 解释器难以同时向多个 GPU 高效发送运算指令，因此某些 GPU 可能处于等待状态。
- 全局解释器锁（GIL）： Python 存在 GIL 限制，多线程并行计算受限，不能充分并行地执行 Python 代码，进一步加剧了 CPU 端的调度瓶颈。

In [6]:
import torch
from torch import nn
from d2l import torch as d2l

# 生产网络的工厂模式
def get_net():
    net = nn.Sequential(nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 2))
    return net

x = torch.randn(size=(1, 512))
net = get_net()
net(x)

tensor([[-0.0093,  0.1486]], grad_fn=<AddmmBackward0>)

In [5]:
net = torch.jit.script(net)
net(x)

tensor([[ 0.0855, -0.1013]], grad_fn=<AddmmBackward0>)

torch.jit.script 是 PyTorch 提供的一种将 Python 代码转换为 TorchScript 的工具，它能把动态图转换为静态图表示。其主要作用和优化原理如下：

- 转换为静态图：
torch.jit.script 会分析和编译 Python 中的模型代码，将其转换为 TorchScript，这是一种中间表示形式。转换后的代码结构固定、可静态分析，从而不需要依赖 Python 解释器逐行解释执行。

- 性能优化：
静态图允许编译器对计算图进行全局优化，例如图融合、内存规划和运行时高效调度，减少了 Python 解释器的开销。编译后的 TorchScript 模块可直接在 C++ 运行时执行，从而提高执行效率，也便于跨平台部署。

- 调试和部署：
转换后的代码更容易进行跨平台部署（如在没有 Python 环境的服务器上运行），同时也支持序列化和延迟执行。

为了证明通过编译获得了性能改进，我们比较了混合编程前后执行net(x)所需的时间。让我们先定义一个度量时间的类，它在本章中在衡量（和改进）模型性能时将非常有用。

In [7]:
#@save
class Benchmark:
    """用于测量运行时间"""
    def __init__(self, description='Done'):
        self.description = description

    def __enter__(self):
        self.timer = d2l.Timer()
        return self

    def __exit__(self, *args):
        print(f'{self.description}: {self.timer.stop():.4f} sec')

现在我们可以调用网络两次，一次使用torchscript，一次不使用torchscript

In [8]:
net = get_net()
with Benchmark('无torchscript'):
    for i in range(1000): net(x)

net = torch.jit.script(net)
with Benchmark('有torchscript'):
    for i in range(1000): net(x)

无torchscript: 0.2103 sec
有torchscript: 0.1721 sec


编译模型的好处之一是我们可以将模型及其参数序列化（保存）到磁盘。这允许这些训练好的模型部署到其他设备上，并且还能方便地使用其他前端编程语言。同时，通常编译模型的代码执行速度也比命令式编程更快。让我们看看save的实际功能

In [10]:
net.save('my_mlp')
!ls -lh my_mlp*

'ls' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
